<a href="https://colab.research.google.com/github/MatheusPiassiC/redes_neurais/blob/main/k_means_hardcore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementação hardcore do algoritmo k-means
O objetivo deste código é realizar uma implementação manual do algorítmo k-means, sem utilizar bibliotecas que o implementem. Além disso, todo o tratamento dos dados também será feito de forma manual e sem utilizar implementações prontas. Os dados usados serão os do dataset Iris, e o nosso algoritmo deve clusterizar cada entrada.

##Tratamento dos dados
A primeira etapa é analisar os dados que usaremos no algoritmo. Para facilitar, o dataset será importado diretamente da biblioteca *Sklearn*.

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

iris = datasets.load_iris() #guarda apenas os dados numéricos, ou seja, sem a coluna de labels

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
X = df.sample(frac=1, random_state=13).reset_index(drop=True) #embaralha o dataframe


print(X.head(10))

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.7               2.8                4.5               1.3
1                5.6               2.9                3.6               1.3
2                5.1               3.7                1.5               0.4
3                5.8               2.7                5.1               1.9
4                6.8               3.0                5.5               2.1
5                5.5               4.2                1.4               0.2
6                7.7               3.0                6.1               2.3
7                6.3               2.7                4.9               1.8
8                5.1               3.8                1.6               0.2
9                5.5               2.6                4.4               1.2


Depois disso, o próximo passo é escalonar os dados para que todos os atributos tenham a mesma escala. Para isso, implementaremos uma função para descobrir o mínimo e o máximo do dataframe, e depois aplicaremos escalonamento de fato.

In [ ]:
def calcular_min_max_params(X: pd.DataFrame): #descobre o min-max de um conjunto
    params = {}
    for col in X.columns:
        min_val = X[col].min()
        max_val = X[col].max()
        params[col] = (min_val, max_val)
    return params

def aplicar_min_max_scaling(X: pd.DataFrame, params: dict) -> pd.DataFrame: #normalização usando mim-max
    X_scaled = X.copy()
    for col, (min_val, max_val) in params.items():
        X_scaled[col] = (X[col] - min_val) / (max_val - min_val)
    return X_scaled

min_max = calcular_min_max_params(X)
X = aplicar_min_max_scaling(X, min_max)
print(X.head(10))
print(X.shape)

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0           0.388889          0.333333           0.593220          0.500000
1           0.361111          0.375000           0.440678          0.500000
2           0.222222          0.708333           0.084746          0.125000
3           0.416667          0.291667           0.694915          0.750000
4           0.694444          0.416667           0.762712          0.833333
5           0.333333          0.916667           0.067797          0.041667
6           0.944444          0.416667           0.864407          0.916667
7           0.555556          0.291667           0.661017          0.708333
8           0.222222          0.750000           0.101695          0.041667
9           0.333333          0.250000           0.576271          0.458333
(150, 4)


##Classe K-Means
A próxima estapa é criar uma classe para o k-means, para facilitar a implementação e ajudar na organização do código.

In [ ]:
class K_Means_Manual:
  def __init__(self, k: int = 3, max_iter: int = 100):
    self.k = k
    self.max_iter = max_iter

    self.centroids = np.array([])
    self.labels = np.array([])

  def inicializar_centroids(self, X:pd.DataFrame):
    np.random.seed(13)
    indices = np.random.choice(X.shape[0], self.k, replace = False) #escolhe k indices no dataframe para serem os primeiros centroides
    return X.iloc[indices] #retorna os centroides

  def atribuir_cluster(self, X: pd.DataFrame):
    ks = self.centroids.shape[0]
    distancias = np.zeros((X.shape[0], ks))

    for j in range(ks): #calcula a distância euclidiana de cada instância em X em relação aos centroides
      diferenca = X - self.centroids.iloc[j]
      distancias[:,j] = np.linalg.norm(diferenca, axis=1) #preenche todas as linhas da coluna j com a distância euclidiana

    clusters = np.argmin(distancias, axis=1)
    return clusters

  def recalcular_centroides(self, X: pd.DataFrame, X_clusters: pd.DataFrame):
    novos_centroides = np.zeros((self.k, X.shape[1]))
    for j in range(self.k):
      novos_centroides[j] = X[X_clusters == j].mean(axis=0)
    return novos_centroides

  def calcular_inercia(self, X, clusters):
    inercia = 0
    for j in range(self.k):
        pontos_cluster = X[clusters == j]
        if len(pontos_cluster) > 0:
            distancias = np.linalg.norm(pontos_cluster - self.centroids.iloc[j].values, axis=1)
            inercia += np.sum(distancias ** 2)
    return inercia

  def treinar(self, X: pd.DataFrame):
    self.centroids = self.inicializar_centroids(X)
    for i in range(self.max_iter):
        X_clusters = self.atribuir_cluster(X)
        novos_centroides = self.recalcular_centroides(X, X_clusters)

        if np.allclose(self.centroids, novos_centroides):
            print("Convergência alcançada na iteração:", i+1)
            break

        self.centroids = pd.DataFrame(novos_centroides, columns=X.columns)
        print("Distribuição dos pontos por cluster:", np.bincount(X_clusters))

    # --- métricas finais ---
    self.labels = self.atribuir_cluster(X)
    inercia = self.calcular_inercia(X.values, self.labels)

    print("\nRelatório de Convergência:")
    print(f"- Iterações necessárias: {i+1}")
    print(f"- Inércia final (Soma dos Erros Quadráticos): {inercia:.4f}")
    print("\nCentroides finais (por feature):")
    print(self.centroids.round(4))

  def prever(self, X: pd.DataFrame):
    X_clusters = self.atribuir_cluster(X)
    return X_clusters

##Resultados + plotagem do gráfico 3D usando PCA

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

def plot_clusters_pca_3d_interativo(X, clusters, centroids):
    # Reduz para 3 dimensões com PCA
    pca = PCA(n_components=3)
    X_pca = pca.fit_transform(X)
    centroids_pca = pca.transform(centroids)

    # Criar DataFrame com clusters
    import pandas as pd
    df = pd.DataFrame(X_pca, columns=["PC1", "PC2", "PC3"])
    df["Cluster"] = clusters

    fig = px.scatter_3d(df, x="PC1", y="PC2", z="PC3",
                        color="Cluster", opacity=0.7)

    # Adicionar centroides em vermelho
    fig.add_scatter3d(
        x=centroids_pca[:,0], y=centroids_pca[:,1], z=centroids_pca[:,2],
        mode="markers", marker=dict(size=10, color="red", symbol="x"),
        name="Centroides"
    )

    fig.show()

print("======= K-means para k = 3 ========")
k_means = K_Means_Manual(k = 3)
centroids = k_means.treinar(X)
clusters_X = k_means.prever(X)
print("Centroids: \n" + str(centroids))
print("Silhouette score: " + str(silhouette_score(X, clusters_X)))
plot_clusters_pca_3d_interativo(X, clusters_X, k_means.centroids)

print("======= K-means para k = 5 ========")
k_means2 = K_Means_Manual(k = 5)
k_means2.treinar(X)
clusters_X2 = k_means2.prever(X)
print("Silhouette score: " + str(silhouette_score(X, clusters_X2)))
plot_clusters_pca_3d_interativo(X, clusters_X2, k_means2.centroids)



======= K-means para k = 3 ========
Distribuição dos pontos por cluster: [53 53 44]
Distribuição dos pontos por cluster: [41 59 50]
Distribuição dos pontos por cluster: [43 57 50]
Distribuição dos pontos por cluster: [45 55 50]
Distribuição dos pontos por cluster: [46 54 50]
Distribuição dos pontos por cluster: [48 52 50]
Convergência alcançada na iteração: 7

Relatório de Convergência:
- Iterações necessárias: 7
- Inércia final (Soma dos Erros Quadráticos): 7.1228

Centroides finais (por feature):
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0             0.4120            0.2769             0.5590            0.5208
1             0.6677            0.4431             0.7572            0.7821
2             0.1961            0.5950             0.0783            0.0608
Centroids: 
None
Silhouette score: 0.4829289335430163


======= K-means para k = 5 ========
Distribuição dos pontos por cluster: [47 53  4 30 16]
Distribuição dos pontos por cluster: [49 51  7 26 17]
Convergência alcançada na iteração: 3

Relatório de Convergência:
- Iterações necessárias: 3
- Inércia final (Soma dos Erros Quadráticos): 5.8621

Centroides finais (por feature):
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0             0.4133            0.2832             0.5597            0.5230
1             0.6716            0.4404             0.7604            0.7851
2             0.3413            0.8512             0.0799            0.0774
3             0.2147            0.6330             0.0847            0.0705
4             0.1078            0.4314             0.0678            0.0392
Silhouette score: 0.37392801075901627


##PCA com 1 e 2 componentes

In [ ]:
def plot_clusters_pca_1d(X, clusters, centroids, titulo="Clusters - PCA 1D"):
    pca = PCA(n_components=1)
    X_pca = pca.fit_transform(X)
    centroids_pca = pca.transform(centroids)

    df = pd.DataFrame(X_pca, columns=["PC1"])
    df["Cluster"] = clusters
    df["PC2"] = 0

    fig = px.scatter(df, x="PC1", y="PC2", color="Cluster",
                     opacity=0.7, title=titulo)

    fig.add_scatter(x=centroids_pca[:,0], y=[0]*len(centroids_pca),
                    mode="markers", marker=dict(size=12, color="red", symbol="x"),
                    name="Centroides")
    fig.show()


def plot_clusters_pca_2d(X, clusters, centroids, titulo="Clusters - PCA 2D"):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    centroids_pca = pca.transform(centroids)

    df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
    df["Cluster"] = clusters

    fig = px.scatter(df, x="PC1", y="PC2", color="Cluster",
                     opacity=0.7, title=titulo)

    fig.add_scatter(x=centroids_pca[:,0], y=centroids_pca[:,1],
                    mode="markers", marker=dict(size=12, color="red", symbol="x"),
                    name="Centroides")
    fig.show()


plot_clusters_pca_1d(X, clusters_X, k_means.centroids)
plot_clusters_pca_2d(X, clusters_X, k_means.centroids)